In [1]:
from sys import path
path +=['..']

In [2]:
from mylib import Puller, PlotDisplayer, Slider, Time
import time
import numpy as np
import pandas as pd
import datetime
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import gridplot, layout
from jupyter_ui_poll import ui_events
import ipywidgets
import matplotlib
import scipy
from scipy import stats
import random
import os

output_notebook()
pl = Puller(simulate=True)  #simulate=True
random.seed()

# инициализация запускать только один раз

Loading BokehJS ...

In [3]:
Time.SetZeroTime()
Time.unfreez()

In [4]:
pl.ms.StopButton() # кнопки для остоновки моторчиков в нештатной ситуации

GridspecLayout(children=(Button(description='Stop All Motors', layout=Layout(grid_area='widget001'), style=But…

In [5]:
pl.ms.motorM.Set_velocity(v=20, a=20)

0

In [16]:
pl.ms.MoveToStart(zapas=10) # сдвиг моторчиков на начальные позиции для растяжки
pl.ms.motorM.MoveTo(17);
while pl.ms.IsInMotion():
    pass

In [7]:
pl.tg.SetZeroWeight(3) # калибровка тензодатчика

9275.785413727697

In [8]:
# pl.sim.l0=pl.ms.Distance(useL0=False)+2

In [9]:
pl.tg.ReadValue(tau=0)

0.0

In [10]:
pl.SetW(20, dw=0.1, tau=1,quiet=False); # натяжка волокна

w=  0.0 , dw=  -20.0 , dx=   0.04623303699500446
w=  16.8279816240738 , dw=  -3.1720183759262 , dx=   0.007332602146151499
w=  19.495859861237477 , dw=  -0.5041401387625228 , dx=   0.00116539648430372
w=  19.919848625380837 , dw=  -0.08015137461916311 , dx=   0.00018528207339841147
SetW finish value= 19.987256357677325


In [11]:
pl.ms.motorM.MoveTo(pl.ms.motorM.position_max-2); # опускание молибдена вниз перед разогревом

In [12]:
pl.tg.SetZeroWeight(2,20) # калибровка тензодатчика после разогрева молибдена

-0.012743642322675441

In [17]:
x0=15
pl.ms.x0=x0
pl.sim.hotFL=True

In [18]:
while pl.ms.IsInMotion():
    pass
dp=PlotDisplayer()
# dp2=PlotDisplayer()
# dp2.CreateMaket(mainParam= 'time',obr_coff=pd.DataFrame({
#     'time': pl.times,
#     'kof': pl.trueKmas
# }))
dp.CreateMaket(mainParam='time',power=pl.data[['time','motorL','motorR']],
                    tension=pl.data[['time','tension','tensionWgl', 'tensionEXPgl']],
                  motors=pl.data[['time','motorR','motorL','motorM']]
                #   v_and_a=pl.data[['time','vL','vR','aL','aR']]
              )
sl=Slider()
sl.NewSl('v',min=0.1,max=9,value=3,step=0.1)
sl.NewSl('a',min=0.1,max=9,value=7,step=0.1)
sl.NewSl('dvKof', max=3,value=0,step=0.01)
sl.NewSl('dv', max=0.5,value=0.001,step=0.0001)
sl.NewSl('window',min=50, max=10000,value=100, step=50)
sl.NewSl('molibdenH',min=-1.6, max=2.0,value=2.0,step=0.01)
sl.NewSl('Kp',min=0, max=10,value=0.25,step=0.01)
sl.NewSl('Ki',min=0, max=10,value=0,step=0.01)
sl.NewSl('Kd',min=0, max=10,value=0,step=0.01)
sl.NewSl('T',min=0, max=200,value=10,step=0.01)
sl.NewBtn('molibden','up molibden','doun molibden')
sl.NewBtn('end','end','end2','end3','vse','bolshe ne jmi','nu pravda ne nado', 'samiy umniy da?')
sl.NewBtn('start dv','start dv','stop dv')
sl.NewBtn('start stabili','start st','stop st')
sl.NewTt('x')
sl.NewTt('L')
sl.NewTt('R')
sl.NewTt('dv')

In [ ]:
pl.MotorsControlStart()
t0 = time.time()
t = t0
dp.Show()
# dp2.Show()
sl.Display()
pl.Clear()

with ui_events() as poll:
    while True:
        poll(10)
        t = time.time()     
        if pl.PulMotorsControl(sl.Sl['molibdenH'](),sl.Sl['T'](),
                                    upFl=sl.BtnFl['molibden'], stFl=sl.BtnFl['end'],
                                    vsFl=sl.BtnFl['start stabili'], dhKof=0.5, ah=9,Kp=sl.Sl['Kp']()*0.001,Ki=sl.Sl['Ki'](),Kd=sl.Sl['Kd']()
                                 )==-1:
            break
        x=pl.data['x'].iloc[-1]
        pl.v=sl.Sl['v']()
        pl.a=sl.Sl['a']()
        T=sl.Sl['T']()
        ww=int(sl.Sl['window']())
        sl.ChangeValueTt('x',x)
        sl.ChangeValueTt('L',pl.ms.L_x(x))
        sl.ChangeValueTt('R',pl.ms.R_x(x))
        sl.ChangeValueTt('dv',pl.dv)    
        dp.Apdate(for_all=pl.data.iloc[-ww:])
#         dp2.Apdate(for_all=pd.DataFrame({
#                     'time': pl.times,
#                     'kof': pl.trueKmas
#                 }))
        push_notebook()    
        Time.sleep(0.04)
print('ura')
pl.Save()
with ui_events() as poll:
    while sl.BtnFl['end']<3:
        poll(10)
        pl.SetW(2,dw=1,tau=0.5)
print('vse')

L= 9.4574


In [9]:
pl.ms.End() # растановка моторчиков на нули (запускать перед отключением питания)

ending
end


In [32]:
pl.Save() # сохранение данных

data saved
